# Model with character recognition - dual model

Builds on `RNN-Morse-chars-feat` but creates two models. One will deal with keying recognition (dits, dahs and silences) then its output will be used to feed a second model dedicated to character recognition. Both models are trained using the same base data but trainings are (of course) done separately.

It looks like this one is working! See explanation on character labeling.

## Create string

Each character in the alphabet should happen a large enough number of times. As a rule of thumb we will take some multiple of the number of characters in the alphabet. If the multiplier is large enough the probability of each character appearance will be even over the alphabet. 

In [ ]:
import MorseGen

morse_gen = MorseGen.Morse()
alphabet = morse_gen.alphabet14
print(132/len(alphabet))

morsestr = MorseGen.get_morse_str(nchars=132*5, nwords=27*5, chars=alphabet)
print(alphabet)
print(len(morsestr), morsestr)

## Generate dataframe and extract envelope

In [ ]:
Fs = 8000
samples_per_dit = morse_gen.nb_samples_per_dit(Fs, 13)
n_prev = int((samples_per_dit/128)*12*2) + 1
print(f'Samples per dit at {Fs} Hz is {samples_per_dit}. Decimation is {samples_per_dit/128:.2f}. Look back is {n_prev}.')
label_df = morse_gen.encode_df_decim_str(morsestr, samples_per_dit, 128, alphabet)
env = label_df['env'].to_numpy()
print(type(env), len(env))

In [ ]:
import numpy as np

def get_new_data(morse_gen, SNR_dB=-23, nchars=132, nwords=27, phrase=None, alphabet="ABC"):
    if not phrase:
        phrase = MorseGen.get_morse_str(nchars=nchars, nwords=nwords, chars=alphabet)
    print(len(phrase), phrase)
    Fs = 8000
    samples_per_dit = morse_gen.nb_samples_per_dit(Fs, 13)
    n_prev = int((samples_per_dit/128)*19) + 1 # number of samples to look back is slightly more than a "O" a word space (2+3+7=12)
    print(f'Samples per dit at {Fs} Hz is {samples_per_dit}. Decimation is {samples_per_dit/128:.2f}. Look back is {n_prev}.')
    label_df = morse_gen.encode_df_decim_tree(phrase, samples_per_dit, 128, alphabet)
    # extract the envelope
    envelope = label_df['env'].to_numpy()
    # remove the envelope
    label_df.drop(columns=['env'], inplace=True)
    SNR_linear = 10.0**(SNR_dB/10.0)
    SNR_linear *= 256 # Apply original FFT
    print(f'Resulting SNR for original {SNR_dB} dB is {(10.0 * np.log10(SNR_linear)):.2f} dB')
    t = np.linspace(0, len(envelope)-1, len(envelope))
    power = np.sum(envelope**2)/len(envelope)
    noise_power = power/SNR_linear
    noise = np.sqrt(noise_power)*np.random.normal(0, 1, len(envelope))
    # noise = butter_lowpass_filter(raw_noise, 0.9, 3) # Noise is also filtered in the original setup from audio. This empirically simulates it
    signal = (envelope + noise)**2
    signal[signal > 1.0] = 1.0 # a bit crap ...
    return envelope, signal, label_df, n_prev

Try it...

In [ ]:
import matplotlib.pyplot as plt 

envelope, signal, label_df, n_prev = get_new_data(morse_gen, SNR_dB=-17, phrase=morsestr, alphabet=alphabet)

# Show
print(n_prev)
print(type(signal), signal.shape)
print(type(label_df), label_df.shape)
    
x0 = 0
x1 = 1500

plt.figure(figsize=(50,4+0.5*len(morse_gen.alphabet)))
plt.plot(signal[x0:x1]*0.7, label="sig")
plt.plot(envelope[x0:x1]*0.9, label='env')
plt.plot(label_df[x0:x1].dit*0.9 + 1.0, label='dit')
plt.plot(label_df[x0:x1].dah*0.9 + 1.0, label='dah')
plt.plot(label_df[x0:x1].ele*0.9 + 2.0, label='ele')
plt.plot(label_df[x0:x1].chr*0.9 + 2.0, label='chr', color="orange")
plt.plot(label_df[x0:x1].wrd*0.9 + 2.0, label='wrd')
plt.plot(label_df[x0:x1].nul*0.9 + 3.0, label='nul')
for i, a in enumerate(alphabet):
    plt.plot(label_df[x0:x1][a]*0.9 + 4.0 + i, label=a)
plt.title("signal and labels")
plt.legend()
plt.grid()

## Create data loader for keying model
### Define keying dataset

In [ ]:
import torch

class MorsekeyingDataset(torch.utils.data.Dataset):
    def __init__(self, morse_gen, device, SNR_dB=-23, nchars=132, nwords=27, phrase=None, alphabet="ABC"):
        self.envelope, self.signal, self.label_df0, self.seq_len = get_new_data(morse_gen, SNR_dB=SNR_dB, phrase=phrase, alphabet=alphabet)
        self.label_df = self.label_df0[['dit','dah','ele','chr','wrd']]
        self.X = torch.FloatTensor(self.signal).to(device)
        self.y = torch.FloatTensor(self.label_df.values).to(device)
        
    def __len__(self):
        return self.X.__len__() - self.seq_len

    def __getitem__(self, index):
        return (self.X[index:index+self.seq_len], self.y[index+self.seq_len])
    
    def get_envelope(self):
        return self.envelope
    
    def get_signal(self):
        return self.signal
    
    def get_labels(self):
        return self.label_df
    
    def get_labels0(self):
        return self.label_df0
    
    def get_seq_len(self):
        return self.seq_len()

### Define keying data loader

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_keying_dataset = MorsekeyingDataset(morse_gen, device, -20, 132*5, 27*5, morsestr, alphabet)
train_keying_loader = torch.utils.data.DataLoader(train_keying_dataset, batch_size=1, shuffle=False) # Batch size must be 1

In [ ]:
signal = train_keying_dataset.get_signal()
envelope = train_keying_dataset.get_envelope()
label_df = train_keying_dataset.get_labels()
label_df0 = train_keying_dataset.get_labels0()

print(type(signal), signal.shape)
print(type(label_df), label_df.shape)

x0 = 0
x1 = 1500

plt.figure(figsize=(50,3))
plt.plot(signal[x0:x1]*0.5, label="sig")
plt.plot(envelope[x0:x1]*0.9, label='env')
plt.plot(label_df[x0:x1].dit*0.9 + 1.0, label='dit')
plt.plot(label_df[x0:x1].dah*0.9 + 1.0, label='dah')
plt.plot(label_df[x0:x1].ele*0.9 + 2.0, label='ele')
plt.plot(label_df[x0:x1].chr*0.9 + 2.0, label='chr')
plt.plot(label_df[x0:x1].wrd*0.9 + 2.0, label='wrd')
plt.title("keying - signal and labels")
plt.legend()
plt.grid()

## Create model classes

The model classes are the same they will be instantiated differently for keying and character models 

### Create model for keying recognition

In [ ]:
import torch
import torch.nn as nn

class MorseLSTM(nn.Module):
    """
    Initial implementation
    """
    def __init__(self, device, input_size=1, hidden_layer_size=8, output_size=6):
        super().__init__()
        self.device = device # This is the only way to get things work properly with device
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1, 1, self.hidden_layer_size).to(self.device),
                            torch.zeros(1, 1, self.hidden_layer_size).to(self.device))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]
    
    def zero_hidden_cell(self):
        self.hidden_cell = (
            torch.zeros(1, 1, self.hidden_layer_size).to(device),
            torch.zeros(1, 1, self.hidden_layer_size).to(device)
        )        
    
class MorseBatchedLSTM(nn.Module):
    """
    Initial implementation
    """
    def __init__(self, device, input_size=1, hidden_layer_size=8, output_size=6):
        super().__init__()
        self.device = device # This is the only way to get things work properly with device
        self.input_size = input_size
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1, 1, self.hidden_layer_size).to(self.device),
                            torch.zeros(1, 1, self.hidden_layer_size).to(self.device))
        self.m = nn.Softmax(dim=-1)

    def forward(self, input_seq):
        #print(len(input_seq), input_seq.shape, input_seq.view(-1, 1, 1).shape)
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(-1, 1, self.input_size), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]
    
    def zero_hidden_cell(self):
        self.hidden_cell = (
            torch.zeros(1, 1, self.hidden_layer_size).to(device),
            torch.zeros(1, 1, self.hidden_layer_size).to(device)
        )     
    
class MorseLSTM2(nn.Module):
    """
    LSTM stack
    """
    def __init__(self, device, input_size=1, hidden_layer_size=8, output_size=6, dropout=0.2):
        super().__init__()
        self.device = device # This is the only way to get things work properly with device
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers=2, dropout=dropout)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(2, 1, self.hidden_layer_size).to(self.device),
                            torch.zeros(2, 1, self.hidden_layer_size).to(self.device))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]
    
    def zero_hidden_cell(self):
        self.hidden_cell = (
            torch.zeros(2, 1, self.hidden_layer_size).to(device),
            torch.zeros(2, 1, self.hidden_layer_size).to(device)
        )        
        
class MorseNoHLSTM(nn.Module):
    """
    Do not keep hidden cell
    """
    def __init__(self, device, input_size=1, hidden_layer_size=8, output_size=6):
        super().__init__()
        self.device = device # This is the only way to get things work properly with device
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, input_seq):
        h0 = torch.zeros(1, 1, self.hidden_layer_size).to(self.device)
        c0 = torch.zeros(1, 1, self.hidden_layer_size).to(self.device)
        lstm_out, _ = self.lstm(input_seq.view(len(input_seq), 1, -1), (h0, c0))
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]
    
class MorseBiLSTM(nn.Module):
    """
    Attempt Bidirectional LSTM: does not work
    """
    def __init__(self, device, input_size=1, hidden_size=12, num_layers=1, num_classes=6):
        super(MorseEnvBiLSTM, self).__init__()
        self.device = device # This is the only way to get things work properly with device
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)  # 2 for bidirection
    
    def forward(self, x):
        # Set initial states
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device) # 2 for bidirection 
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x.view(len(x), 1, -1), (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out[-1]    

Create the keying model instance and print the details

In [ ]:
morse_key_model = MorseBatchedLSTM(device, hidden_layer_size=7, output_size=5).to(device) # This is the only way to get things work properly with device
morse_key_loss_function = nn.MSELoss()
morse_key_optimizer = torch.optim.Adam(morse_key_model.parameters(), lr=0.001)

print(morse_key_model)
print(morse_key_model.device)

In [ ]:
# Input and hidden tensors are not at the same device, found input tensor at cuda:0 and hidden tensor at cpu
for m in morse_key_model.parameters():
    print(m.shape, m.device)
X_t = torch.rand(n_prev)
#X_t = torch.tensor([-0.9648, -0.9385, -0.8769, -0.8901, -0.9253, -0.8637, -0.8066, -0.8066, -0.8593, -0.9341, -1.0000, -0.9385])
X_t = X_t.cuda()
print("Input shape", X_t.shape, X_t.view(-1, 1, 1).shape)
print(X_t)
morse_key_model(X_t)

In [ ]:
import torchinfo
channels=10
H=n_prev
W=1
torchinfo.summary(morse_key_model, input_size=(channels, H, W))

## Train keying model

In [ ]:
it = iter(train_keying_loader)
X, y = next(it)
print(X.reshape(n_prev,1).shape, X[0].shape, y[0].shape)
print(X[0], y[0])
X, y = next(it)
print(X[0], y[0])

In [ ]:
%%time
from tqdm.notebook import tqdm

epochs = 4
morse_key_model.train()

for i in range(epochs):
    train_losses = []
    loop = tqdm(enumerate(train_keying_loader), total=len(train_keying_loader), leave=True)
    for j, train in loop:
        X_train = train[0][0]
        y_train = train[1][0]
        morse_key_optimizer.zero_grad()
        if morse_key_model.__class__.__name__ in ["MorseLSTM", "MorseLSTM2", "MorseBatchedLSTM", "MorseBatchedLSTM2"]:
            morse_key_model.zero_hidden_cell() # this model needs to reset the hidden cell
        y_pred = morse_key_model(X_train)
        single_loss = morse_key_loss_function(y_pred, y_train)
        single_loss.backward()
        morse_key_optimizer.step()
        train_losses.append(single_loss.item())
        # update progress bar
        if j % 1000 == 0:
            loop.set_description(f"Epoch [{i+1}/{epochs}]")
            loop.set_postfix(loss=np.mean(train_losses))

print(f'final: {i+1:3} epochs loss: {np.mean(train_losses):6.4f}')

In [ ]:
save_model = True
if save_model: 
    torch.save(morse_key_model.state_dict(), 'models/morse_key_model')
else:
    morse_key_model.load_state_dict(torch.load('models/morse_key_model', map_location=device))

### Extract results for next step

In [ ]:
from tqdm.notebook import tqdm

p_key_train = torch.empty(1,5).to(device)
morse_key_model.eval()

loop = tqdm(enumerate(train_keying_loader), total=len(train_keying_loader))
for j, train in loop:
    with torch.no_grad():
        X_train = train[0]
        pred_val = morse_key_model(X_train[0])
        p_key_train = torch.cat([p_key_train, pred_val.reshape(1,5)])

In [ ]:
# drop first garbage sample
p_key_train = p_key_train[1:]
print(p_key_train.shape)

In [ ]:
print(p_key_train[0:2])
p_dits = p_key_train[:,0].to('cpu').numpy()
p_dahs = p_key_train[:,1].to('cpu').numpy()
p_eles = p_key_train[:,2].to('cpu').numpy()
p_chrs = p_key_train[:,3].to('cpu').numpy()
p_wrds = p_key_train[:,4].to('cpu').numpy()

plt.figure(figsize=(50,6))
plt.plot(signal[x0+n_prev:x1+n_prev]*0.5, label="sig")
plt.plot(envelope[x0+n_prev:x1+n_prev]*0.9, label='env')
plt.plot(p_dits[x0:x1]*0.9 + 1.0, label='dit')
plt.plot(p_dahs[x0:x1]*0.9 + 1.0, label='dah')
plt.plot(p_eles[x0:x1]*0.9 + 2.0, label='ele', color='orange')
plt.plot(p_chrs[x0:x1]*0.9 + 2.0, label='chr')
plt.plot(p_wrds[x0:x1]*0.9 + 2.0, label='wrd', color='green')
plt.title("keying - predictions")
plt.legend()
plt.grid()

### Post process

  - Take softmax
  - Subtract dit from dah
  - Shift dit, ele, chr and wrd so the bottom is approximately zero

In [ ]:
sm = nn.Softmax(dim=1)
p_sm_train = sm(p_key_train)
p_sm_train[:,1] -= p_sm_train[:,0]
p_sm_train[:,0] = p_sm_train[:,0]*2.0 - 0.25
p_sm_train[:,2] -= 0.125
p_sm_train[:,3] -= 0.125
p_sm_train[:,4] -= 0.125

In [ ]:
p_sm_dits = p_sm_train[:,0].to('cpu').numpy()
p_sm_dahs = p_sm_train[:,1].to('cpu').numpy()
p_sm_eles = p_sm_train[:,2].to('cpu').numpy()
p_sm_chrs = p_sm_train[:,3].to('cpu').numpy()
p_sm_wrds = p_sm_train[:,4].to('cpu').numpy()

plt.figure(figsize=(50,6))
plt.plot(signal[x0+n_prev:x1+n_prev]*0.5, label="sig")
plt.plot(envelope[x0+n_prev:x1+n_prev]*0.9, label='env')
plt.plot(p_sm_dits[x0:x1] + 1.0, label='dit')
plt.plot(p_sm_dahs[x0:x1] + 1.0, label='dah')
plt.plot(p_sm_eles[x0:x1] + 2.0, label='ele', color='orange')
plt.plot(p_sm_chrs[x0:x1] + 2.0, label='chr')
plt.plot(p_sm_wrds[x0:x1] + 2.0, label='wrd', color='green')
plt.title("keying - softmax predictions")
plt.legend()
plt.grid()

## Create data loader for character model

The key to successful decoding lies in following the Morse tree as you would do in procedural programming (see `RNN-Morse-chars-hyb` that already produces significant results). Thie is the "Morse tree":

<img src="files/Morse1Min.gif">

The root of the tree is the "start" state. At each character break the current state is reset at "start" state. Then at each new element (dit or dah) the state moves to the next node on the left (dah) or the right (dit) and the character decoded so far is simply attached to the node. The actual decoded character is the one attached to the node of the current state right before it is reset to "start" by the character separator.

There is one label per character plus a "nul" character for characters that are not recognized by any state (some nodes do not yield any useful character and are just intermediate nodes). To build the labels one simply follows the logic described above. In addition the character label is extended to the next "ele" and "chr" periods. 

To get the final result the character lines are multiplied (gated) by the character separator line ("chr") the most probable character is then the one with maximum value.

### Define character dataset

In [ ]:
class MorseCharacterDataset(torch.utils.data.Dataset):
    def __init__(self, key_train, label_df, seq_len):
        self.label_df = label_df.drop(columns=['dit','dah'])
        self.X = key_train
        self.y = torch.FloatTensor(self.label_df.values).to(device)
        self.seq_len = seq_len
        
    def __len__(self):
        return self.X.__len__() - self.seq_len

    def __getitem__(self, index):
        return (self.X[index:index+self.seq_len], self.y[index+self.seq_len])
    
    def get_X(self):
        return self.X
    
    def get_labels(self):
        return self.label_df
    
    def get_seq_len(self):
        return self.seq_len

### Define character data loader

In [ ]:
train_character_dataset = MorseCharacterDataset(p_sm_train, label_df0[n_prev:].reset_index(drop=True), n_prev)
train_character_loader = torch.utils.data.DataLoader(train_character_dataset, batch_size=1, shuffle=False) # Batch size must be 1

In [ ]:
X_train_chr = train_character_dataset.get_X().cpu()
label_df_chr = train_character_dataset.get_labels()

print(type(X_train_chr), X_train_chr.shape)
print(type(label_df_chr), label_df_chr.shape)

x0 = 0
x1 = 1500

plt.figure(figsize=(50,3+0.5*len(alphabet)))
plt.plot(X_train_chr[x0:x1,0], label='Xdit')
plt.plot(X_train_chr[x0:x1,1], label='Xdah')
plt.plot(X_train_chr[x0:x1,3] + 1.0, label='Xchr')
plt.plot(label_df_chr[x0:x1]['ele']*0.9 + 2.0, label='ele', color="orange")
plt.plot(label_df_chr[x0:x1]['chr']*0.9 + 2.0, label='chr')
plt.plot(label_df_chr[x0:x1]['wrd']*0.9 + 2.0, label='wrd')
for i, a in enumerate(alphabet):
    plt.plot(label_df_chr[x0:x1][a]*0.9 + 3.0 + i, label=a)
plt.title("character - signal and labels")
plt.legend(loc=2)
plt.grid()

## Create model for character recognition

In [ ]:
morse_chr_model = MorseBatchedLSTM(device, input_size=5, hidden_layer_size=len(alphabet)*2, output_size=len(alphabet)+4).to(device) # This is the only way to get things work properly with device
morse_chr_loss_function = nn.MSELoss()
morse_chr_optimizer = torch.optim.Adam(morse_chr_model.parameters(), lr=0.001)

print(morse_chr_model)
print(morse_chr_model.device)

In [ ]:
# Input and hidden tensors are not at the same device, found input tensor at cuda:0 and hidden tensor at cpu
for m in morse_chr_model.parameters():
    print(m.shape, m.device)
X_t = torch.rand(n_prev, 5)
#X_t = torch.tensor([-0.9648, -0.9385, -0.8769, -0.8901, -0.9253, -0.8637, -0.8066, -0.8066, -0.8593, -0.9341, -1.0000, -0.9385])
X_t = X_t.cuda()
print("Input shape", X_t.shape, X_t.view(-1, 1, 5).shape)
morse_chr_model(X_t)
# Does not work...

In [ ]:
channels=10
H=n_prev
W=5
torchinfo.summary(morse_chr_model, input_size=(channels, H, W))

## Train character model

In [ ]:
it = iter(train_character_loader)
X, y = next(it)
print(X.reshape(n_prev,5).shape, X[0].shape, y[0].shape)
print(y[0])

In [ ]:
epochs = 4
morse_chr_model.train()

for i in range(epochs):
    train_losses = []
    loop = tqdm(enumerate(train_character_loader), total=len(train_character_loader), leave=True)
    for j, train in loop:
        X_train = train[0][0]
        y_train = train[1][0]
        morse_chr_optimizer.zero_grad()
        if morse_chr_model.__class__.__name__ in ["MorseLSTM", "MorseLSTM2", "MorseBatchedLSTM", "MorseBatchedLSTM2"]:
            morse_chr_model.zero_hidden_cell() # this model needs to reset the hidden cell
        y_pred = morse_chr_model(X_train)
        single_loss = morse_chr_loss_function(y_pred, y_train)
        single_loss.backward()
        morse_chr_optimizer.step()
        train_losses.append(single_loss.item())
        # update progress bar
        if j % 1000 == 0:
            loop.set_description(f"Epoch [{i+1}/{epochs}]")
            loop.set_postfix(loss=np.mean(train_losses))

print(f'final: {i+1:3} epochs loss: {np.mean(train_losses):6.4f}')

In [ ]:
save_model = True
if save_model: 
    torch.save(morse_key_model.state_dict(), 'models/morse_char_model')
else:
    morse_key_model.load_state_dict(torch.load('models/morse_char_model', map_location=device))

In [ ]:
%%time
p_char_train = torch.empty(1,18).to(device)
morse_chr_model.eval()

loop = tqdm(enumerate(train_character_loader), total=len(train_character_loader))
for j, train in loop:
    with torch.no_grad():
        X_chr = train[0][0]
        pred_val = morse_chr_model(X_chr)
        p_char_train = torch.cat([p_char_train, pred_val.reshape(1,18)])        

In [ ]:
p_char_train = p_char_train[1:] # Remove garbge
print(p_char_train.shape) # t -> chars(t)

### Post process
  
  - Move to CPU to ger chars(time)
  - Transpose to get times(char)

In [ ]:
p_char_train_c = p_char_train.cpu() # t -> chars(t) on CPU
p_char_train_t = torch.transpose(p_char_train_c, 0, 1).cpu() # c -> times(c) on CPU
print(p_char_train_c.shape, p_char_train_t.shape)

In [ ]:
l_alpha = label_df_chr[n_prev:].reset_index(drop=True)
plt.figure(figsize=(50,4+0.5*len(morse_gen.alphabet)))
plt.plot(l_alpha[x0:x1]["chr"]*(len(alphabet)+1)+2, label="ychr", alpha=0.2, color="black")
plt.plot(X_train_chr[x0+n_prev:x1+n_prev, 0]*1.9, label='dit')
plt.plot(X_train_chr[x0+n_prev:x1+n_prev, 1]*1.9, label='dah')
plt.plot(X_train_chr[x0+n_prev:x1+n_prev, 2]*1.9 + 1.0, label='ele')
plt.plot(X_train_chr[x0+n_prev:x1+n_prev, 3]*1.9 + 1.0, label='chr')
plt.plot(X_train_chr[x0+n_prev:x1+n_prev, 4]*1.9 + 1.0, label='wrd')
plt.plot(p_char_train_t[1][x0:x1]*0.9 + 2.0, label='c')
plt.plot(p_char_train_t[2][x0:x1]*0.9 + 2.0, label='w')
for i, a in enumerate(alphabet):
    plt_a = plt.plot(p_char_train_t[i+4][x0:x1]*0.9 + 3.0 + i, label=a)
    plt.plot(l_alpha[a][x0:x1]*0.5 + 3.0 + i, color=plt_a[0].get_color(), alpha=0.5)
plt.title("predictions")
plt.legend(loc=2)
plt.grid()

## Test

### Test keying dataset and data loader

In [ ]:
teststr = "AAAA USERS ARE USING EGGS AND GRAIN MONGO TEST MADAME WONDER WOMAN GOOD MAMA USSR WAS GREAT AAA"
test_keying_dataset = MorsekeyingDataset(morse_gen, device, -17, 132*5, 27*5, teststr, alphabet)
test_keying_loader = torch.utils.data.DataLoader(test_keying_dataset, batch_size=1, shuffle=False) # Batch size must be 1

### Run the keying model

In [ ]:
p_key_test = torch.empty(1,5).to(device)
morse_key_model.eval()

loop = tqdm(enumerate(test_keying_loader), total=len(test_keying_loader))
for j, test in loop:
    with torch.no_grad():
        X_test = test[0]
        pred_val = morse_key_model(X_test[0])
        p_key_test = torch.cat([p_key_test, pred_val.reshape(1,5)])

In [ ]:
# drop first garbage sample
p_key_test = p_key_test[1:]
print(p_key_test.shape)

### Post process

In [ ]:
p_sm_test = sm(p_key_test)
p_sm_test[:,1] -= p_sm_test[:,0]
p_sm_test[:,0] = p_sm_test[:,0]*2.0 - 0.25
p_sm_test[:,2] -= 0.125
p_sm_test[:,3] -= 0.125
p_sm_test[:,4] -= 0.125

In [ ]:
print(p_key_test[0:2])
p_dits_t = p_sm_test[:,0].to('cpu').numpy()
p_dahs_t = p_sm_test[:,1].to('cpu').numpy()
p_eles_t = p_sm_test[:,2].to('cpu').numpy()
p_chrs_t = p_sm_test[:,3].to('cpu').numpy()
p_wrds_t = p_sm_test[:,4].to('cpu').numpy()

signal_t = test_keying_dataset.get_signal()
envelope_t = test_keying_dataset.get_envelope()
label_df0_t = test_keying_dataset.get_labels0()

plt.figure(figsize=(50,6))
plt.plot(signal_t[x0+n_prev:x1+n_prev]*0.5, label="sig")
plt.plot(envelope_t[x0+n_prev:x1+n_prev]*0.9, label='env')
plt.plot(p_dits_t[x0:x1]*0.9 + 1.0, label='dit')
plt.plot(p_dahs_t[x0:x1]*0.9 + 1.0, label='dah')
plt.plot(p_eles_t[x0:x1]*0.9 + 2.0, label='ele', color='orange')
plt.plot(p_chrs_t[x0:x1]*0.9 + 2.0, label='chr')
plt.plot(p_wrds_t[x0:x1]*0.9 + 2.0, label='wrd', color='green')
plt.title("keying - predictions")
plt.legend()
plt.grid()

### Test character dataset and data loader

In [ ]:
test_character_dataset = MorseCharacterDataset(p_sm_test, label_df0_t[n_prev:].reset_index(drop=True), n_prev)
test_character_loader = torch.utils.data.DataLoader(test_character_dataset, batch_size=1, shuffle=False) # Batch size must be 1

### Run the character model

In [ ]:
%%time
p_char_test = torch.empty(1,18).to(device)
morse_chr_model.eval()

loop = tqdm(enumerate(test_character_loader), total=len(test_character_loader))
for j, test in loop:
    with torch.no_grad():
        X_chr = test[0][0]
        pred_val = morse_chr_model(X_chr)
        p_char_test = torch.cat([p_char_test, pred_val.reshape(1,18)])  

In [ ]:
p_char_test = p_char_test[1:] # Remove garbge
print(p_char_test.shape) # t -> chars(t)

In [ ]:
p_char_test_c = p_char_test.cpu() # t -> chars(t) on CPU
p_char_test_t = torch.transpose(p_char_test_c, 0, 1).cpu() # c -> times(c) on CPU
print(p_char_test_c.shape, p_char_test_t.shape)

### Show results

In [ ]:
X_test_chr = test_character_dataset.get_X().cpu()
label_df_chr_t = test_character_dataset.get_labels()
l_alpha_t = label_df_chr_t[n_prev:].reset_index(drop=True)

#### Raw results

In [ ]:
plt.figure(figsize=(100,4+0.5*len(morse_gen.alphabet)))
plt.plot(l_alpha_t[:]["chr"]*(len(alphabet)+1)+2, label="ychr", alpha=0.2, color="black")
plt.plot(X_test_chr[n_prev:, 0]*1.9, label='dit')
plt.plot(X_test_chr[n_prev:, 1]*1.9, label='dah')
plt.plot(X_test_chr[n_prev:, 2]*1.9 + 1.0, label='ele')
plt.plot(X_test_chr[n_prev:, 3]*1.9 + 1.0, label='chr')
plt.plot(X_test_chr[n_prev:, 4]*1.9 + 1.0, label='wrd')
plt.plot(p_char_test_t[1]*0.9 + 2.0, label='c', color="green")
plt.plot(p_char_test_t[2]*0.9 + 2.0, label='w', color="red")
for i, a in enumerate(alphabet):
    plt_a = plt.plot(p_char_test_t[i+4,:]*0.9 + 3.0 + i, label=a)
    plt.plot(l_alpha_t[a]*0.5 + 3.0 + i, color=plt_a[0].get_color(), linestyle="--")
plt.title("predictions")
plt.legend(loc=2)
plt.grid()
plt.savefig('img/predicted.png')

#### Gated by character prediction

In [ ]:
plt.figure(figsize=(200,4+0.5*len(morse_gen.alphabet)))
plt.plot(l_alpha_t["chr"]*(len(alphabet)+1)+2, label="ychr", alpha=0.2, color="black")
plt.plot(X_test_chr[n_prev:, 0], label='dit')
plt.plot(X_test_chr[n_prev:, 1], label='dah')
plt.plot(X_test_chr[n_prev:, 2] + 1.0, label='ele')
plt.plot(X_test_chr[n_prev:, 3] + 1.0, label='chr')
plt.plot(X_test_chr[n_prev:, 4] + 1.0, label='wrd')
plt.plot(p_char_test_t[1]*0.9 + 2.0, label="cp", color="green")
plt.plot(p_char_test_t[2]*0.9 + 2.0, label="wp", color="red")
for i, a in enumerate(alphabet):
    line_a = p_char_test_t[i+4] * p_char_test_t[1]
    plt_a = plt.plot(line_a*0.9 + 3.0 + i, label=a)
    plt.plot(l_alpha_t[a]*0.5 + 3.0 + i, color=plt_a[0].get_color(), linestyle="--")
plt.title("predictions")
plt.legend(loc=2)
plt.grid()
plt.savefig('img/predicted_gated.png')

## Procedural decision making

In [ ]:
class MorseDecoder:
    def __init__(self, alphabet, chr_len, wrd_len):
        self.nb_alpha = len(alphabet)
        self.alphabet = alphabet
        self.chr_len = chr_len
        self.wrd_len = wrd_len
        self.sums = [0.0 for x in range(self.nb_alpha+2)]
        self.tests = [0.0 for x in range(self.nb_alpha+2)]
        self.prevs = [0.0 for x in range(self.nb_alpha+2)]
        self.counts = [0 for x in range(self.nb_alpha+2)]
        self.res = ""

    def new_samples(self, samples):
        for i, s in enumerate(samples):
            if i > 2:
                t = s * samples[0] # gating for alpha characters
            else:
                t = s
            if i > 0:
                j = i-1
                if t >= 0.5 and self.prevs[j] < 0.5:
                    self.counts[j] = 0
                if t > 0.5:
                    self.sums[j] = self.sums[j] + t
                    self.tests[j] = 0.0
                else:
                    blk_len = wrd_len if j == 0 else chr_len
                    if self.counts[j] > blk_len:
                        self.tests[j] = self.sums[j]
                        self.sums[j] = 0.0
                self.counts[j] += 1
                self.prevs[j] = t
        if np.sum(self.tests) > 0.0:
            ci = np.argmax(self.tests)
            if ci == 0:
                self.res += " "
            elif ci > 1:
                self.res += self.alphabet[ci - 2]

In [ ]:
chr_len = round(samples_per_dit*2 / 128)
wrd_len = round(samples_per_dit*4 / 128)
decoder = MorseDecoder(alphabet, chr_len, wrd_len)
for s in p_char_test_c:
    decoder.new_samples(s[1:]) # c, w, n, [alpha]
print(decoder.res)

In [ ]:
round(samples_per_dit*4 / 128)

In [ ]:
test_l = p_char_test_t[1]
test_sum = 0.0
test_count = 0
int_count = 0
prev_avg = None
res = []
prev_avg = None
for t in test_l:
    if prev_avg is None:
        prev_avg = t
        test_sum = t
        test_count = 1
        continue
    proxy_avg = (test_sum + t) / (test_count + 1)
    if proxy_avg >= prev_avg:
        avg = proxy_avg
        test_count += 1
        int_count = 0
    else:
        avg = t
        prev_avg = 0
        test_sum = t
        test_count = 1
        int_count += 1
    if int_count < 5:
        res.append(avg)
    else:
        res.append(0)
    prev_avg = avg
                
plt.figure(figsize=(300,6))
plt.plot(res, color="red")
plt.plot(test_l, color="green")
plt.grid()

In [ ]:
char_l = p_char_test_t[1]
test_l = p_char_test_t[5]
test_sum = 0.0
res = []
gat = []
for i, t in enumerate(test_l):
    s = t * char_l[i]
    gat.append(s)
    if s > 0.5:
        test_sum = test_sum + s
        res.append(0)
    else:
        res.append(test_sum)
        test_sum = 0.0
                
plt.figure(figsize=(300,6))
plt.plot(res, color="red")
plt.plot(test_l, color="green")
plt.plot(gat, color="orange")
plt.grid()